In [2]:
import pandas as pd

In [3]:
def CleanData(dataFrame:pd.DataFrame):
    #Make sure what was passed in is a dataframe so we can use functions like drop and get_dummies
    if type(dataFrame) != pd.DataFrame:
        raise RuntimeError("that is not a data frame")
    dataFrame.drop(columns={"Datasource", "Data_Value_Unit", "GeoLocation", "Data_Value_Type", "Low_Confidence_Limit", "Low_Confidence_Limit"}, inplace=True)
    return pd.DataFrame(dataFrame)

In [4]:
df = pd.read_csv("Nutrition__Physical_Activity__and_Obesity_-_Behavioral_Risk_Factor_Surveillance_System.csv")
df = CleanData(df)
df.sample(10)
#print(df.Class.value_counts(), df.Topic.value_counts())

,YearStart,YearEnd,LocationAbbr,LocationDesc,Class,Topic,Question,Data_Value,Data_Value_Alt,Data_Value_Footnote_Symbol,...,Race/Ethnicity,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
5696,2012,2012,DC,District of Columbia,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,30.0,30.0,NaN,...,NaN,OWS,OWS1,Q037,VALUE,11,Total,Total,OVR,OVERALL
38267,2015,2015,AL,Alabama,Physical Activity,Physical Activity - Behavior,Percent of adults who achieve at least 300 min...,24.6,24.6,NaN,...,NaN,PA,PA1,Q045,VALUE,1,Age (years),35 - 44,AGEYR,AGEYR3544
43270,2015,2015,OK,Oklahoma,Physical Activity,Physical Activity - Behavior,Percent of adults who achieve at least 150 min...,52.1,52.1,NaN,...,NaN,PA,PA1,Q043,VALUE,40,Income,"$50,000 - $74,999",INC,INC5075
17438,2013,2013,MO,Missouri,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in muscle-strengt...,32.6,32.6,NaN,...,NaN,PA,PA1,Q046,VALUE,29,Age (years),25 - 34,AGEYR,AGEYR2534
24604,2012,2012,OH,Ohio,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,18.3,18.3,NaN,...,NaN,PA,PA1,Q047,VALUE,39,Income,"$50,000 - $74,999",INC,INC5075
33245,2011,2011,WV,West Virginia,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming fruit l...,NaN,NaN,~,...,Other,FV,FV1,Q018,VALUE,54,Race/Ethnicity,Other,RACE,RACEOTH
44741,2015,2015,VA,Virginia,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in muscle-strengt...,24.6,24.6,NaN,...,NaN,PA,PA1,Q046,VALUE,51,Age (years),65 or older,AGEYR,AGEYR65PLUS
32348,2013,2013,VA,Virginia,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in muscle-strengt...,26.3,26.3,NaN,...,NaN,PA,PA1,Q046,VALUE,51,Gender,Female,GEN,FEMALE
9771,2012,2012,IN,Indiana,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,31.7,31.7,NaN,...,NaN,OWS,OWS1,Q036,VALUE,18,Income,"$25,000 - $34,999",INC,INC2535
40434,2015,2015,IA,Iowa,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,21.0,21.0,NaN,...,NaN,PA,PA1,Q047,VALUE,19,Income,"$50,000 - $74,999",INC,INC5075


In [5]:
print(df.columns)

Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'Class',
       'Topic', 'Question', 'Data_Value', 'Data_Value_Alt',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'Low_Confidence_Limit', 'High_Confidence_Limit ', 'Sample_Size',
       'Total', 'Age(years)', 'Education', 'Gender', 'Income',
       'Race/Ethnicity', 'ClassID', 'TopicID', 'QuestionID', 'DataValueTypeID',
       'LocationID', 'StratificationCategory1', 'Stratification1',
       'StratificationCategoryId1', 'StratificationID1'],
      dtype='object')


In [6]:
df["High_Confidence_Limit "].value_counts()

37.1    222
37.8    218
37.9    217
37.2    212
35.6    209
       ... 
73.1      1
73.0      1
7.7       1
5.7       1
8.7       1
Name: High_Confidence_Limit , Length: 723, dtype: int64

In [28]:
df["StratificationCategory1"].value_counts()

Race/Ethnicity    15256
Income            13349
Age (years)       11438
Education          7628
Gender             3814
Total              1907
Name: StratificationCategory1, dtype: int64

In [37]:
dfRace = df[df["StratificationCategory1"] == "Race/Ethnicity"]
dfIncome = df[df["StratificationCategory1"] == "Income"]
dfAge = df[df["StratificationCategory1"] == "Age (years)"]
dfEducation = df[df["StratificationCategory1"] == "Education"]
dfGender = df[df["StratificationCategory1"] == "Gender"]
dfTotals = df[df["StratificationCategory1"] == "Total"]

In [38]:
dfTotals.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,Class,Topic,Question,Data_Value,Data_Value_Alt,Data_Value_Footnote_Symbol,...,Race/Ethnicity,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
0,2011,2011,AL,Alabama,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,32.0,32.0,NaN,...,NaN,OWS,OWS1,Q036,VALUE,1,Total,Total,OVR,OVERALL
28,2011,2011,AL,Alabama,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,34.7,34.7,NaN,...,NaN,OWS,OWS1,Q037,VALUE,1,Total,Total,OVR,OVERALL
56,2011,2011,AL,Alabama,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming fruit l...,43.8,43.8,NaN,...,NaN,FV,FV1,Q018,VALUE,1,Total,Total,OVR,OVERALL
84,2011,2011,AL,Alabama,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming vegetab...,24.3,24.3,NaN,...,NaN,FV,FV1,Q019,VALUE,1,Total,Total,OVR,OVERALL
113,2011,2011,AL,Alabama,Physical Activity,Physical Activity - Behavior,Percent of adults who achieve at least 150 min...,42.4,42.4,NaN,...,NaN,PA,PA1,Q043,VALUE,1,Total,Total,OVR,OVERALL
